# PART 0: INSTALL (for Colab)

In [2]:
!pip install cma

     |████████████████████████████████| 230 kB 1.4 MB/s eta 0:00:01


# PART 1: EVALUATION

In [4]:
import cma
from cma.fitness_transformations import EvalParallel2

## TOY EXAMPLE: ROSENBROCK FUNCTION

In [5]:
# TOY EXAMPLE: ROSENBROCK FUNCTION
es = cma.CMAEvolutionStrategy(8 * [0], 0.5)

f = cma.ff.rosen
#es.optimize(f)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=130092, Tue Nov 10 20:57:39 2020)


In [10]:
with EvalParallel2(f, number_of_processes=12) as eval:
    while not es.stop():
        X = es.ask()
        es.tell(X, eval(X))
print(es.result[1], es.result[2])

5.488439864559804e-15 4000


# PART 2: EVAL FUNCTION WRAPPER

In [ ]:
def launch_run(hyperparameters):
    """
        TODO: write training run code for ASR
        TODO: integrate with ESPnet and feature extraction model training procedures

        Generate a training run of ASR system with selected hyperparameters
        :param hyperparameters: 1D array of hyperparameter values
        :return: evaluation metric
    """  
    raise NotImplementedError